## PAIR PROGRAMMING ETL I

### Extracción -API's
---

In [56]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np

Vamos a nutrir los datos de los ataques de los tiburones con información climática de los paises que tenemos. Para eso vamos a usar la API del clima que hemos aprendido en la clase invertida:

In [8]:
#url = f'http://www.7timer.info/bin/api.pl?lon=-{longitud}&lat={latitud}&product={producto}&output=json'

En este caso os recomendamos que uséis el producto meteo para obtener la información climática. Para hacer la llamada a la API necesitamos también las coordenadas de los paises que tenemos en el dataset.

Por lo tanto, el objetivo es que saquéis la información del clima de la API para los paises que tenemos. Pero antes de poneros manos a la obra, tenemos muchísimos países y esto puede hacerse eterno. Sacad la información climática solo para los siguientes países, con las siguientes coordenadas:

In [57]:
pais = ['USA', 'Australia', 'South Africa', 'New Zealand', 'Papua New Guinea']
latitud= [39.7837304, -24.7761086, -28.8166236, -41.5000831, -5.6816069]
longitud = [-100.445882, 134.755, 24.991639,172.8344077,144.2489081]

producto = 'meteo'

![alt text]('../anamor/ADALAB/PairProgramming/imagenpairETL.png')

Requisitos de este ejercicio de pair programming:
- Deberéis meter toda la información climática en un único dataframe.
- Deberéis hacer la llamada a la API de una sola vez. Es decir, tendréis que iterar por la lista de países y sacar la información del clima para cada uno de ellos.
- Al meter toda la información en un único dataframe tendremos que crear una columna que corresponda con el nombre del país.
- Guardar el dataframe obtenido para usarlo en la siguiente sesión de pair programming.

💡 Pistas 💡:
- Crearos un diccionario donde:
    - Las keys sean los nombres de los paises
    - Los values sean las coordenadas de los paises en formato lista.

- Iterar por el diccionario. Dentro del for haremos la llamada a la API como hemos aprendido en las clases invertidas. Recordamos que para iterar por el diccionario tenemos que seguir la siguiente estructura: 
*_for key, value in diccionario.items(): 
print(key, value)_*

- Crearnos un dataframe vacío donde iremos concatenando los datos obtenidos de la API cada país.
- Guardad el csv ya que mañana lo necesitaremos para hacer el ejercicio de pair programming.

In [10]:
diccionario = list(zip(pais,longitud,latitud))# reveer si podemos sacarlo con el zip para no hacerlo de forma manual
diccionario

[('USA', -100.445882, 39.7837304),
 ('Australia', 134.755, -24.7761086),
 ('South Africa', 24.991639, -28.8166236),
 ('New Zealand', 172.8344077, -41.5000831),
 ('Papua New Guinea', 144.2489081, -5.6816069)]

In [61]:
dicc_paises= {'usa': [-100.445882, 39.7837304],'australia': [134.755, -24.7761086],'south africa': [24.991639, -28.8166236],'new zealand': [172.8344077, -41.5000831],'papua new guinea': [144.2489081, -5.6816069]}

In [62]:
clima_paises = pd.DataFrame( columns= ['timepoint', 'cloudcover', 'lifted_index', 'prec_type', 'prec_amount','temp2m', 'rh2m', 'weather', 'wind10m.direction', 'wind10m.speed','country'])

for key, value in dicc_paises.items(): 

   

    url =f'http://www.7timer.info/bin/api.pl?lon=-{dicc_paises[key][0]}&lat={dicc_paises[key][1]}&product={producto}&output=json'


    response = requests.get(url=url)

    response.status_code


    response.reason

    df = pd.json_normalize(response.json()['dataseries'])

    df['country']=key

    clima_paises=pd.concat([clima_paises,df], axis=0, ignore_index = True)



200

'OK'

200

'OK'

200

'OK'

200

'OK'

200

'OK'

In [63]:
clima_paises.head()

,timepoint,cloudcover,lifted_index,prec_type,prec_amount,temp2m,rh2m,weather,wind10m.direction,wind10m.speed,country,highcloud,midcloud,lowcloud,rh_profile,wind_profile,msl_pressure,snow_depth
0,3,1,15,none,0,12,4,NaN,270,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",1026.0,0.0
1,6,1,15,none,0,12,3,NaN,15,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",1027.0,0.0
2,9,1,15,none,0,12,2,NaN,310,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",1028.0,0.0
3,12,1,15,none,0,14,1,NaN,60,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 270, 'speed':...",1028.0,0.0
4,15,1,15,none,0,14,3,NaN,85,2,usa,-9999.0,-9999.0,-9999.0,"[{'layer': '950mb', 'rh': 1}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 180, 'speed':...",1027.0,0.0


In [64]:
clima_paises['country'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

In [65]:
clima_paises.shape

(320, 18)

In [66]:
clima_paises.to_csv('../files/clima_paises.csv')